# About this workbook
***
In the previous examples I've shown how to create a clustering model for customer segmentation as well as demonstrated how to implement association analysis to mine commonly purchased items. In this workbook I'll show you how to combine both of these approaches in order to create a sales strategy.

In [20]:
# import the required python libraries for analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder
import warnings 
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = "{:,.2f}".format

In [3]:
# Set seaborn styles
sns.set(font_scale=1.6)
sns.set_style(style='whitegrid',
              rc={'axes.spines.left': True
                 ,'axes.spines.bottom': True
                 ,'axes.spines.top': False
                 ,'axes.spines.bottom': True
                 ,'axes.spines.right': False
                 ,'axes.spines.left': True
                 ,'axes.facecolor': 'white'
                 ,'axes.edgecolor': 'lightgray'
                 ,'ytick.left': True
                 ,'xtick.bottom': True
                 ,'grid.linestyle': '--'
                 ,'font.family': ['Arial']})

In [4]:
# read in dataset. This is stored in my local repo but is not tracked in the remote repository.
# the "../" is to go back two folders since I store the data in the root of my local repository.
data = pd.read_csv("../../data/Groceries_dataset.csv")

In [5]:
# Update column names
data.columns = ['member_id', 'date', 'item_desc']

# Update date column data type
# I have to use to_datetime twice since the date is stored as dd-mm-yyyy
data['date'] = pd.to_datetime(pd.to_datetime(data["date"]).dt.strftime('%Y-%m-%d'))

In [6]:
# Create a week & month variable
data['date_week'] = pd.to_datetime(data["date"]).dt.to_period('W-SUN').dt.start_time
data['date_month'] = data['date'] - pd.tseries.offsets.MonthBegin(1)

### Clustering (Customer Segmentation)

In [10]:
# Aggregate the data per member id
cluster_df = (
    data.groupby(['member_id'], as_index=False)
    .agg(
        first_purchase=('date', 'min'),
        last_purchase=('date', 'max'),
        frequency=('item_desc', 'count'),
        unique_weeks=('date_week', 'nunique'),
        unique_months=('date_month', 'nunique'),
        items=('item_desc', 'nunique')
    )
)

# Create some additional variables
cluster_df['age'] = (cluster_df['last_purchase'] - cluster_df['first_purchase']).dt.days
cluster_df['times_per_week'] = cluster_df['frequency'] / cluster_df['unique_weeks']
cluster_df['times_per_month'] = cluster_df['frequency'] / cluster_df['unique_months']
cluster_df['product_breadth'] = cluster_df['items'] / data['item_desc'].nunique()
cluster_df['frequency_log'] = np.log(cluster_df['frequency'])

In [11]:
x = cluster_df[['frequency_log', 'age', 'product_breadth']]

scaler = StandardScaler().fit(x)
x_scaled = scaler.transform(x)

In [12]:
# Construct the clustering model
model = KMeans(n_clusters=5, random_state=42).fit(x_scaled)

In [22]:
score = silhouette_score(x_scaled, model.labels_)
print(f"kmeans Cluster Score: {score:0.2%}")

kmeans Cluster Score: 36.06%


In [13]:
# Assign the cluster labels back to the cluster dataframe
# Then only select the member_id and cluster index
cluster_df['customer_group'] = model.labels_
cluster_df = cluster_df[['member_id', 'customer_group']]

In [15]:
# See a sample of the clustered data
cluster_df.sample(5, random_state=9)

,member_id,customer_group
540,1553,0
3495,4593,3
1060,2086,4
2103,3163,1
503,1514,0


In [16]:
# join back with the original dataset
data = data.merge(cluster_df, how='left', on='member_id')

In [17]:
data.head()

,member_id,date,item_desc,date_week,date_month,customer_group
0,1808,2015-07-21,tropical fruit,2015-07-20,2015-07-01,3
1,2552,2015-05-01,whole milk,2015-04-27,2015-04-01,0
2,2300,2015-09-19,pip fruit,2015-09-14,2015-09-01,0
3,1187,2015-12-12,other vegetables,2015-12-07,2015-12-01,2
4,3037,2015-01-02,whole milk,2014-12-29,2015-01-01,1


### Combined Association Analysis
In this section we're going to leverage the clustered groups to create frequent item sets per group.

In [23]:
# Make sure to strip the item desription column of any white space
data['item_desc'] = data['item_desc'].str.strip()

In [24]:
# Create an invoice column. If you have invoices in your system this step is not neccessary.
data['invoice'] = data['date'].astype(str) + '-' + data['member_id'].astype(str)

In [25]:
# This function create the list of transactions
def build_associations(data):
    basket = (
        data.groupby('invoice')['item_desc']
        .apply(list).reset_index()
    )
    transactions = basket.item_desc.tolist()
    
    # Set up the transaction encoder and encode the transactions
    te = TransactionEncoder()
    te_array = te.fit(transactions).transform(transactions)
    encoded = pd.DataFrame(te_array, columns = te.columns_)
    
    # Build the frequently purchased items
    freq_items = fpgrowth(encoded, min_support=0.001, use_colnames=True)
    
    # Build the associations
    rules = association_rules(freq_items, metric='lift', min_threshold=0.5)
    
    return rules 

In [26]:
# write a loop that build the association rules per group
# Only run this for group 0 and 4 since they were identified as our key target customers
associations_dict = {}
for group in [0, 4]:
    loop_label = "Group {}".format(group)
    
    # Run the associations method
    rules = build_associations(data[data['customer_group']==group]) 
    
    # Store results in python dictionary
    associations_dict[loop_label] = rules
    

In [29]:
group_1 = associations_dict.get('Group 0')
group_5 = associations_dict.get('Group 4')

#### Group 1 Associations

In [32]:
group_1.sort_values(by='confidence', ascending=False)[:5]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
620,"(soda, margarine)",(whole milk),0.00,0.16,0.00,0.40,2.49,0.00,1.40
680,(house keeping products),(whole milk),0.00,0.16,0.00,0.38,2.33,0.00,1.34
134,"(yogurt, sausage)",(whole milk),0.01,0.16,0.00,0.29,1.78,0.00,1.17
965,(liquor (appetizer)),(soda),0.00,0.09,0.00,0.27,3.04,0.00,1.25
622,"(margarine, whole milk)",(soda),0.00,0.09,0.00,0.27,3.04,0.00,1.25


In [33]:
# Show rules that don't contain milk
group_1[
    (~group_1['consequents'].astype(str).str.contains('milk'))
    & (~group_1['antecedents'].astype(str).str.contains('milk'))
].sort_values(by='confidence', ascending=False)[:5]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
965,(liquor (appetizer)),(soda),0.00,0.09,0.00,0.27,3.04,0.00,1.25
1132,(cling film/bags),(rolls/buns),0.00,0.11,0.00,0.24,2.22,0.00,1.17
109,"(rolls/buns, sausage)",(other vegetables),0.00,0.12,0.00,0.23,1.89,0.00,1.14
7,"(soda, rolls/buns)",(other vegetables),0.01,0.12,0.00,0.20,1.64,0.00,1.10
110,"(other vegetables, sausage)",(rolls/buns),0.01,0.11,0.00,0.20,1.85,0.00,1.11


#### Group 5 Associations

In [34]:
group_5.sort_values(by='confidence', ascending=False)[:5]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1584,"(citrus fruit, frankfurter)",(whole milk),0.00,0.16,0.00,0.71,4.33,0.00,2.92
573,"(dessert, yogurt)",(whole milk),0.00,0.16,0.00,0.50,3.03,0.00,1.67
153,"(shopping bags, yogurt)",(whole milk),0.00,0.16,0.00,0.44,2.65,0.00,1.48
1280,"(pork, sausage)",(whole milk),0.00,0.16,0.00,0.42,2.53,0.00,1.43
1820,"(rolls/buns, beef)",(whole milk),0.00,0.16,0.00,0.42,2.53,0.00,1.43


In [35]:
# Show rules that don't contain milk
group_5[
    (~group_5['consequents'].astype(str).str.contains('milk'))
    & (~group_5['antecedents'].astype(str).str.contains('milk'))
].sort_values(by='confidence', ascending=False)[:5]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1129,"(whipped/sour cream, yogurt)",(rolls/buns),0.00,0.11,0.00,0.38,3.43,0.00,1.44
1580,"(frankfurter, bottled beer)",(other vegetables),0.00,0.13,0.00,0.36,2.79,0.00,1.36
1928,(dog food),(yogurt),0.00,0.09,0.00,0.33,3.66,0.00,1.36
1128,"(whipped/sour cream, rolls/buns)",(yogurt),0.00,0.09,0.00,0.33,3.66,0.00,1.36
429,"(tropical fruit, pastry)",(rolls/buns),0.00,0.11,0.00,0.31,2.79,0.00,1.29
